[View in Colaboratory](https://colab.research.google.com/github/sekharvth/sentiment-analysis-lstm/blob/master/Question_Classifier.ipynb)

In [3]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving answers_6000.txt to answers_6000.txt
Saving cleaned_questions_minus_w to cleaned_questions_minus_w
User uploaded file "answers_6000.txt" with length 380176 bytes
User uploaded file "cleaned_questions_minus_w" with length 286755 bytes


In [7]:
import os 
os.listdir('/content')

['datalab',
 '.local',
 '.forever',
 '.cache',
 'answers_6000.txt',
 'vocab',
 '.config',
 '.rnd',
 'glove.6B.zip',
 'cleaned_questions_minus_w',
 '.keras',
 'content',
 'negative_instances_AMI.txt',
 'positive_instances_AMI.txt',
 '.ipython']

In [5]:
!wget http://nlp.stanford.edu/data/glove.6B.zip && unzip glove.6B.zip -d content
 

--2018-04-26 06:37:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-04-26 06:37:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip         67%[============>       ] 558.65M  8.15MB/s    eta 14s    

glove.6B.zip        100%[===================>] 822.24M  10.9MB/s    in 60s     

2018-04-26 06:38:37 (13.6 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: content/glove.6B.50d.txt  
  inflating: content/glove.6B.100d.txt  
  inflating: content/glove.6B.200d.txt  
  inflating: content/glove.6B.300d.txt  


In [0]:
import numpy as np
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
  

words_to_index, index_to_words, word_to_vec_map = read_glove_vecs('content/glove.6B.300d.txt')

In [0]:
import os
import pandas as pd
#!pip install -q keras
import keras

In [0]:

import numpy as np
import re
import itertools
from collections import Counter


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #common = set(string.split()).intersection(person_list)
    #if len(common) > 0:
    #    for i in list(common):
    #        if i in ['ms', 'mrs', 'mr', 'dr']:
    #            string = re.sub(r'{} ?[A-Za-z]+ '.format(i), 'person', string)
    #        else:
    #            string = string.replace(i, ' person ')
    #print (string, '\n')
    string = string.strip().lower()
    #common = set(sent.split()).intersection(lst)
    #if len(common) > 0:
    #    for i in list(common):
    #        if i in ['ms', 'mrs', 'mr', 'dr']:
    #            string = re.sub(r'{} ?[A-Za-z]+ '.format(i), 'person ', string)
    #        else:
    #            string = string.replace(i, ' person ')
    #print (string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " have", string)
    if 'wont ' in string:
      string = re.sub(r"n\'?t", " will not", string)
    if 'won\'t ' in string:
      string = re.sub(r"n\'?t", " will not", string)
    
    if 'cant ' in string:
      string = re.sub(r"n\'?t", " can not", string)
    if 'can\'t ' in string:
      string = re.sub(r"n\'?t", " can not", string)

    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    return string.strip()

def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = pd.read_csv(positive_data_file, sep = '\n', header = None)[0]
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = pd.read_csv(negative_data_file, sep = '\n', header = None)[0]
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]




In [0]:
x_text, y = load_data_and_labels('cleaned_questions_minus_w', 'answers_6000.txt')

In [0]:
import tensorflow as tf
vocab_processor1 = tf.contrib.learn.preprocessing.VocabularyProcessor.restore('vocab')

In [12]:

import tensorflow as tf
#maxlen = len(max(x_text_new, key = len))
#for pos,i in enumerate(x_text_new):
#  x_text_new[pos] = i + [0] * maxlen
#  x_text_new[pos] = x_text_new[pos][:maxlen]
  
#for pos,i in enumerate(x_test_new):
#  x_test_new[pos] = i + [0] * maxlen
#  x_test_new[pos] = x_test_new[pos][:maxlen]
maxlen = len(max(x_text, key = len).split())
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(maxlen)
x = np.array(list(vocab_processor.transform(x_text)))
vocab_dictionary = vocab_processor.vocabulary_._mapping
vocab_size = len(vocab_dictionary)
#x_test_new = np.array(x_test_new)
#x_text_new = np.array(x_text_new)

#vocab_size =  len(words_to_index)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [13]:
len(vocab_dictionary)

5511

In [0]:
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(0.2 * float(len(x)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

sample_index = -1 * int(0.5 * float(len(x_dev)))
x_dev, x_test = x_dev[:sample_index], x_dev[sample_index:]
y_dev, y_test = y_dev[:sample_index], y_dev[sample_index:]

del x, y, x_shuffled, y_shuffled


In [15]:

# import necessary packages
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, GRU
from keras.layers.embeddings import Embedding
np.random.seed(1)

num_classes = 2

embed_matrix = np.zeros((vocab_size, 300))
for word,index in vocab_dictionary.items():
  try:
    embed_matrix[index, :] = word_to_vec_map[word]
  except:
    embed_matrix[index, :] = np.random.uniform(-1, 1, 300)
    

# make a Keras embedding layer of shape (vocab_size, emb_dim) and set 'trainable' argument to 'True' if you want to train your
# own word embeddings on top of pre trained GloVe vectors(can improve performance, as we'll get embeddings more suited to current text corpus)
embed_layer = Embedding(input_dim = vocab_size, output_dim = 300, embeddings_initializer = 'glorot_uniform', trainable = True)

embed_layer.build([None])

embed_layer.set_weights([embed_matrix])


# Start defining the Keras model
# Define sentence_indices as the input to the model, of shape (maxlen,) and dtype 'int32'
sentence_indices = Input(shape = (maxlen,), dtype = 'int32')

# Propagate sentence_indices through the embedding layer, to get the embeddings
embeddings = embed_layer(sentence_indices)  

# Propagate the embeddings through an LSTM layer with 300-dimensional hidden state
X = GRU(300, return_sequences = True)(embeddings)

# Add dropout with a probability of 0.5
X = Dropout(0.5)(X)

# Propagate X trough another LSTM layer with 300-dimensional hidden state
X = GRU(300)(X)

# Add dropout with a probability of 0.5
X = Dropout(0.5)(X)

# Propagate X through a Dense layer with softmax activation to get back a batch of 'num_classes' number of values
X = Dense(num_classes, activation = 'softmax')(X)

# create model instance 
model = Model(inputs = sentence_indices, outputs = X)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# fit the training data on the model, while shuffling the order of the training set
model.fit(x_train, y_train, epochs = 10, batch_size = 128, validation_data = (x_dev, y_dev),shuffle=True)


Train on 8388 samples, validate on 1049 samples
Epoch 1/10
8388/8388 [==============================] - 51s 6ms/step - loss: 0.9015 - acc: 0.5154 - val_loss: 0.7391 - val_acc: 0.5529
Epoch 2/10
8388/8388 [==============================] - 50s 6ms/step - loss: 0.7253 - acc: 0.5407 - val_loss: 0.6571 - val_acc: 0.6330
Epoch 3/10
8388/8388 [==============================] - 51s 6ms/step - loss: 0.5555 - acc: 0.7186 - val_loss: 0.4518 - val_acc: 0.8227
Epoch 4/10
8388/8388 [==============================] - 50s 6ms/step - loss: 0.3435 - acc: 0.8689 - val_loss: 0.2820 - val_acc: 0.9028
Epoch 5/10
1408/8388 [====>.........................] - ETA: 40s - loss: 0.2763 - acc: 0.9034

8388/8388 [==============================] - 50s 6ms/step - loss: 0.2626 - acc: 0.9121 - val_loss: 0.3017 - val_acc: 0.8942
Epoch 6/10
8388/8388 [==============================] - 50s 6ms/step - loss: 0.2010 - acc: 0.9345 - val_loss: 0.3136 - val_acc: 0.9171
Epoch 7/10
8388/8388 [==============================] - 50s 6ms/step - loss: 0.1656 - acc: 0.9475 - val_loss: 0.2654 - val_acc: 0.9209
Epoch 8/10
8388/8388 [==============================] - 50s 6ms/step - loss: 0.1264 - acc: 0.9598 - val_loss: 0.3041 - val_acc: 0.8970
Epoch 9/10
4864/8388 [================>.............] - ETA: 20s - loss: 0.1002 - acc: 0.9718

8388/8388 [==============================] - 51s 6ms/step - loss: 0.1062 - acc: 0.9684 - val_loss: 0.2689 - val_acc: 0.9161
Epoch 10/10
8388/8388 [==============================] - 50s 6ms/step - loss: 0.0841 - acc: 0.9750 - val_loss: 0.2663 - val_acc: 0.9256


In [16]:
labels = [np.argmax(i) for i in y_test]
x_test[0]

array([155, 136, 136,  10,  25, 527, 528, 572,  33, 326,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [0]:

all_preds = [np.argmax(i) for i in model.predict(x_test)]

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print ('precision: ', precision_score(labels, all_preds))
print ('recall: ', recall_score(labels, all_preds))
print ('f1 :', f1_score(labels, all_preds))
print ('accuracy :', accuracy_score(labels, all_preds))
confusion_matrix(labels, all_preds)

precision:  0.9093023255813953
recall:  0.8967889908256881
f1 : 0.9030023094688222
accuracy : 0.9198473282442748


array([[573,  39],
       [ 45, 391]])

In [0]:
sen = clean_str('Siri Siri, you want to excel person, one thing and then that is supposed to be the next item right')
sen = sen.strip().split()
for pos,i in enumerate(sen):
    try:
        sen[pos] = vocab_dictionary[i.lower()]
    except:
        sen[pos] = vocab_dictionary['<UNK>']

In [0]:
sen = sen + [0] * 180

In [0]:
sen = np.array(sen[:180]).reshape(1,180)

In [31]:
np.argmax(model.predict(sen))

1

In [0]:
model.save('/content/questions_10_epochs')

In [34]:
from google.colab import files
files.download('questions_10_epochs')

MessageError: ignored